<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./10_CAMS_European_air_quality_forecast_retrieve.ipynb"><< 10 - CAMS European air quality forecast - Nitrogen Dioxide - Retrieve</a><span style="float:right;"><a href="./20_Sentinel5P_TROPOMI_NO2_L2_retrieve.ipynb">20 - Sentinel-5P NO<sub>2</sub> - Retrieve >></a></span>

<div class="alert alert-block alert-warning">
<b>LOAD, BROWSE AND VISUALIZE</b></div>

# CAMS - European Air Quality Forecasts and Analyses - Nitrogen Dioxide

This notebooks provides an introduction to the CAMS European Air Quality Forecasts and Analyses and how the data can be used to monitor increased levels of `Nitrogen Dioxide` over Europe.

The notebook shows the Nitrogen Dioxide forecast from 4 February 2021 for the next three days. During this period, elevated Nitrogen Dioxide values were recorded. CAMS provides maps of daily average surface concentrations of key air pollutants, e.g. NO<sub>2</sub>, PM2.5, PM10 and Ozone [here](https://atmosphere.copernicus.eu/european-air-quality-information-support-covid-19-crisis).

CAMS European Air Quality Forecasts and Analyses are available in either `GRIB` or `netCDF`. You can retrieve the data from the [Atmosphere Data Store](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) or retrieve the data from WEkEO with the help of the HDA API.

<br>

#### Module outline:
* [1 - Load and browse the CAMS Europeaen Air Quality forecast data](#load_browse_europe)
* [2 - Retrieve the data variable Nitrogen Dioxide as xarray.DataArray](#data_retrieve_europ)
* [3 - Visualize Nitrogen Dioxide at surface over Europe](#visualize_europe)
* [5 - Animate Nitroge Dioxide levels at surface over time](#animate_europe)

<hr>

#### Load required libraries

In [ ]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc
import pandas as pd

from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.cm import get_cmap
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
from datetime import datetime

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

#### Load helper functions

In [ ]:
%run ./functions.ipynb

<hr>

### <a id='load_browse_europe'></a>Load and browse `CAMS European Air Quality forecast and analyses` data

CAMS global near-real-time forecast data is available either in `GRIB` or `netCDF`. The data for the present example has been downloaded in the `netCDF` format. The notebook [10_CAMS_European_air_quality_forecast_retrieve](./10_CAMS_European_air_quality_forecast_retrieve.ipynb) shows an example how to access the data with the `WEkEO HDA API`. If data aren't available on the machine, the user may use this notebook to download them. 

You can use xarray's function `xr.open_dataset()` to open the netCDF file as `xarray.Dataset`.

In [ ]:
file = xr.open_dataset('./adaptor.cams_regional_fc.retrieve-1639385759.2822495-18756-2-f1fd6140-71d3-46d4-969c-9fa9002e1b41.nc')
file


<br>

The data above has four dimensions (`latitude`, `longitude`, `level` and `time`) and one data variable:
* `no2_conc`: Mass concentration of Nitrogen Dioxide in the air


Let us inspect the coordinates of the file more in detail. Below, you see that the data set consists of 12 time steps, starting on 4 February 2021 00 UTC and ranging up to 3 days ahead. However, the format of the time coordinates is in nanoseconds.

Let us convert the time information into a human-readable time format.

In [ ]:
file.time

First, from the `long_name` information of the time dimension, we can retrieve the initial timestamp. With the function `strptime()` from Python's `datetime` library, we can convert it into a `datetime.datetime` object.

In [ ]:
timestamp = file.time.long_name[19:27]

timestamp_init=datetime.strptime(timestamp,'%Y%m%d' )
timestamp_init

<br>

In a next step, we then build a `DateTimeIndex` object with the help of Panda's `date_range()` function, making use of the length of time dimension.
The result is a `DateTimeIndex` object, which can be used to newly assign the time coordinate information.

In [ ]:
time_coords = pd.date_range(timestamp_init, periods=len(file.time), freq='6h').strftime("%Y-%m-%d %H:%M:%S").astype('datetime64[ns]')
time_coords


<br>

And the last step is to assign the converted time information to the DataArray `no2_fc`, with the function `assign_coords()`.

In [ ]:
no2_fc = file.assign_coords(time=time_coords)
no2_fc

<br>

The latitude values have a 0.1 degrees resolution and have a European coverage.

In [ ]:
file.latitude

The longitude values have a 0.1 degrees resolution as well, but are on a [0,360] grid instead of a [-180,180] grid. 

In [ ]:
file.longitude

<br>

### <a id='shift_europe'></a>Bring longitude coordinates onto a [-180,180] grid

You can assign new values to coordinates in an `xarray.Dataset`. You can do so with the `assign_coords()` function, which you can apply onto a `xarray.Dataset`. The code below shifts your longitude grid from [0,360] to [-180,180]. At the end, you sort the longitude values in an ascending order.

In [ ]:
file_assigned = no2_fc.assign_coords(longitude=(((no2_fc.longitude + 180) % 360) - 180)).sortby('longitude')
file_assigned

<br>

A quick check of the longitude coordinates of the new `xarray.Dataset` shows you that the longitude values are now on a [-180, 180] longitude grid.

In [ ]:
file_assigned.longitude

### <a id='data_retrieve_europe'></a>Retrieve the data variable `dust` as xarray.DataArray

Let us store the data variable `no2_conc` as `xarray.DataArray` with the name `no2_conc`.

In [ ]:
no2_conc = file_assigned.no2_conc
no2_conc

<br>

Above, you see that the variable `no2_conc` has four attributes, `species`, `units`, `value` and `standard_name`. Let us define variables for the attributes `unit` and `standard_name`. The variables can be used for visualizing the data.

In [ ]:
long_name = no2_conc.standard_name
units = no2_conc.units

<br>

Let us do the same for the coordinates `longitude` and `latitude`.

In [ ]:
latitude = no2_conc.latitude
longitude = no2_conc.longitude

<br>

### <a id='visualize_europe'></a>Visualize `NO2` concentration over Europe

The next step is to visualize the dataset. You can use the function [visualize_pcolormesh](../functions.ipynb#visualize_pcolormesh), which makes use of matploblib's function `pcolormesh` and the [Cartopy](https://scitools.org.uk/cartopy/docs/latest/) library.

With `?visualize_pcolormesh` you can open the function's docstring to see what keyword arguments are needed to prepare your plot.

In [ ]:
?visualize_pcolormesh

<br>

You can make use of the variables we have defined above:
- `units`
- `long_name`
- `latitude`
- `longitude`

Additionally, you can specify the color scale and minimum and maxium data values.

In [ ]:
visualize_pcolormesh(data_array=no2_conc[6,0,:,:],
                     longitude=longitude,
                     latitude=latitude,
                     projection=ccrs.PlateCarree(),
                     color_scale='Reds',
                     unit=units,
                     long_name=long_name + ' ' + str(no2_conc[6,0,:,:].time.data),
                     vmin=0, 
                     vmax=35,
                     lonmin=longitude.min().data,
                     lonmax=longitude.max().data,
                     latmin=latitude.min().data,
                     latmax=latitude.max().data,
                     set_global=False)

<br>

### <a id='animate_europe'></a>Animate `NO2 concentration` forecast over Europe

In the last step, you can animate the `Nitrogen Dioxide concentration` forecast in order to see how the pollutant is predicted over a period of three days, from 4th to 6th February 2021.

You can do animations with matplotlib's function `animation`. Jupyter's function `HTML` can then be used to display HTML and video content.

The animation function consists of 4 parts:
- **Setting the initial state:**<br>
 Here, you define the general plot your animation shall use to initialise the animation. You can also define the number of frames (time steps) your animation shall have.
 
 
- **Functions to animate:**<br>
 An animation consists of three functions: `draw()`, `init()` and `animate()`. `draw()` is the function where individual frames are passed on and the figure is returned as image. In this example, the function redraws the plot for each time step. `init()` returns the figure you defined for the initial state. `animate()` returns the `draw()` function and animates the function over the given number of frames (time steps).
 
 
- **Create a `animate.FuncAnimation` object:** <br>
 The functions defined before are now combined to build an `animate.FuncAnimation` object.
 
 
- **Play the animation as video:**<br>
 As a final step, you can integrate the animation into the notebook with the `HTML` class. You take the generate animation object and convert it to a HTML5 video with the `to_html5_video` function

In [ ]:
# Setting the initial state:
# 1. Define figure for initial plot
fig, ax = visualize_pcolormesh(data_array=no2_conc[0,0,:,:],
                               longitude=no2_conc.longitude, 
                               latitude=no2_conc.latitude,
                               projection=ccrs.PlateCarree(), 
                               color_scale='Reds', 
                               unit=units,
                               long_name=long_name + ' '+ str(no2_conc.time[0].data),
                               vmin=0,
                               vmax=35, 
                               lonmin=longitude.min().data, 
                               lonmax=longitude.max().data, 
                               latmin=latitude.min().data, 
                               latmax=latitude.max().data,
                               set_global=False)

frames = 13

def draw(i):
    img = plt.pcolormesh(no2_conc.longitude, 
                         no2_conc.latitude, 
                         no2_conc[i,0,:,:], 
                         cmap='Reds', 
                         transform=ccrs.PlateCarree(),
                         vmin=0,
                         vmax=35,
                         shading='auto')
    
    ax.set_title(long_name + ' '+ str(no2_conc.time[i].data), fontsize=20, pad=20.0)
    return img


def init():
    return fig


def animate(i):
    return draw(i)

ani = animation.FuncAnimation(fig, animate, frames, interval=800, blit=False,
                              init_func=init, repeat=True)

HTML(ani.to_html5_video())
plt.close(fig)

<br>

#### Play the animation video as HTML5 video

In [ ]:
HTML(ani.to_html5_video())

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./10_CAMS_European_air_quality_forecast_retrieve.ipynb"><< 10 - CAMS European air quality forecast - Nitrogen Dioxide - Retrieve</a><span style="float:right;"><a href="./20_Sentinel5P_TROPOMI_NO2_L2_retrieve.ipynb">20 - Sentinel-5P NO<sub>2</sub> - Retrieve >></a></span>

<hr>

<p><img src='./img/all_partners_wekeo.png' align='left' alt='Logo EU Copernicus' width='100%'></img><p>